<a href="https://colab.research.google.com/github/ChoiGwibin/2-/blob/main/%EC%88%98%EB%8F%84_%EB%9D%BC%EB%B2%A8%EB%A7%81(%EC%95%99%EC%83%81%EB%B8%94)%2C_TTA%2C_%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리 및 환경 설정

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 라이브러리 및 환경 설정
!pip install pandas matplotlib seaborn Pillow tqdm torchmetrics albumentations ultralytics pycocotools PyYAML

import os
import json
import torch
import pandas as pd
import numpy as np
import cv2
import re
import yaml
import shutil
import glob
import seaborn as sns
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import torchvision
from PIL import Image
from tqdm import tqdm
from collections import Counter
from zipfile import ZipFile
from torchvision import transforms as T
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.6 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# 데이터 준비

In [ ]:
!pip install kaggle

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = 'wonhu1'
os.environ['KAGGLE_KEY'] = 'bc6670ea96ac607125d9b0d289990f91'

# Drive 내에서 데이터 다운로드
!kaggle competitions download -c ai04-level1-project -p /content/drive/MyDrive/pill_project/
!cd /content/drive/MyDrive/pill_project && unzip ai04-level1-project.zip

ai04-level1-project.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  ai04-level1-project.zip
replace test_images/1.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import os

# Drive 내 데이터 구조 확인
drive_base = '/content/drive/MyDrive/pill_project'

print("=== Drive 내 데이터 확인 ===")
print(f"프로젝트 폴더: {os.listdir(drive_base)}")

# train_images 확인
train_images_dir = os.path.join(drive_base, 'train_images')
if os.path.exists(train_images_dir):
    image_count = len([f for f in os.listdir(train_images_dir) if f.endswith('.png')])
    print(f"train_images: {image_count}개 이미지")
else:
    print("train_images 폴더가 없습니다.")

# train_annotations 확인
train_annotations_dir = os.path.join(drive_base, 'train_annotations')
if os.path.exists(train_annotations_dir):
    json_files = []
    for root, dirs, files in os.walk(train_annotations_dir):
        json_files.extend([f for f in files if f.endswith('.json')])
    print(f"train_annotations: {len(json_files)}개 JSON 파일")
else:
    print("train_annotations 폴더가 없습니다.")

# test_images 확인
test_images_dir = os.path.join(drive_base, 'test_images')
if os.path.exists(test_images_dir):
    test_count = len([f for f in os.listdir(test_images_dir) if f.endswith('.png')])
    print(f"test_images: {test_count}개 이미지")

=== Drive 내 데이터 확인 ===
프로젝트 폴더: ['ai04-level1-project.zip', 'pill_dataset', 'test_images', 'train_annotations', 'train_images', 'pill_dataset.yaml', 'yolov8n.pt', 'ensemble_models', 'yolov8s.pt', 'pill_dataset_clean', 'pill_clean.yaml', 'ensemble_clean']
train_images: 1489개 이미지
train_annotations: 4526개 JSON 파일
test_images: 843개 이미지


# 데이터 전처리

In [ ]:
# Drive 경로용 수정된 데이터 로드 함수

import os
import json
import glob
import pandas as pd
import cv2
from tqdm import tqdm

def load_all_images_and_annotations(json_dir, image_dir):
    """
    모든 이미지를 로드하고, 어노테이션 유무를 구분 (Drive 경로용)
    """
    # 1. JSON 파일에서 어노테이션 데이터 수집
    json_files = glob.glob(os.path.join(json_dir, '**', '*.json'), recursive=True)
    print(f"총 {len(json_files)}개의 JSON 파일을 찾았습니다.")

    annotated_data = []
    image_info_dict = {}  # 이미지 정보 저장용

    for json_file in tqdm(json_files, desc="JSON 파일 처리 중"):
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)

        if not data.get('images'):
            continue

        image_info = data['images'][0]
        image_id = image_info['id']
        file_name = image_info['file_name']
        image_path = os.path.join(image_dir, file_name)

        # 이미지 기본 정보 저장
        image_info_dict[file_name] = {
            'image_id': image_id,
            'image_path': image_path,
            'file_name': file_name,
            'width': image_info['width'],
            'height': image_info['height'],
            'dl_name': image_info['dl_name'],
            'drug_shape': image_info.get('drug_shape'),
            'color_class1': image_info.get('color_class1'),
            'line_front': image_info.get('line_front'),
        }

        # 어노테이션이 있는 경우
        if 'annotations' in data and data['annotations']:
            for anno in data['annotations']:
                if 'bbox' in anno and anno['bbox'] and len(anno['bbox']) == 4:
                    annotated_data.append({
                        **image_info_dict[file_name],  # 이미지 기본 정보
                        'bbox': anno['bbox'],
                        'category_id': anno['category_id'],
                        'area': anno['area'],
                        'has_annotation': True
                    })

    # 2. 전체 이미지 파일 목록 가져오기
    all_image_files = glob.glob(os.path.join(image_dir, '*.png'))
    all_image_names = [os.path.basename(f) for f in all_image_files]

    # 3. 어노테이션이 없는 이미지들 찾기
    annotated_image_names = set(image_info_dict.keys())
    unlabeled_image_names = set(all_image_names) - annotated_image_names

    print(f"전체 이미지: {len(all_image_names)}장")
    print(f"어노테이션 있는 이미지: {len(annotated_image_names)}장")
    print(f"어노테이션 없는 이미지: {len(unlabeled_image_names)}장")

    # 4. 어노테이션 없는 이미지들도 DataFrame에 추가 (bbox 없이)
    unlabeled_data = []
    for img_name in tqdm(unlabeled_image_names, desc="어노테이션 없는 이미지 처리"):
        img_path = os.path.join(image_dir, img_name)
        if os.path.exists(img_path):
            # 이미지 크기 읽기
            img = cv2.imread(img_path)
            height, width = img.shape[:2] if img is not None else (640, 640)

            unlabeled_data.append({
                'image_id': None,  # ID 없음
                'image_path': img_path,
                'file_name': img_name,
                'width': width,
                'height': height,
                'dl_name': None,  # 약품 이름 없음
                'drug_shape': None,
                'color_class1': None,
                'line_front': None,
                'bbox': None,  # bbox 없음
                'category_id': None,
                'area': None,
                'has_annotation': False  # 어노테이션 없음을 표시
            })

    # 5. 모든 데이터 합치기
    all_data = annotated_data + unlabeled_data
    df_all = pd.DataFrame(all_data)

    print(f"총 {len(all_data)}개의 데이터를 로드했습니다.")
    print(f"- 어노테이션 있음: {len(annotated_data)}개")
    print(f"- 어노테이션 없음: {len(unlabeled_data)}개")

    return df_all

# Drive 경로 설정
DRIVE_BASE = '/content/drive/MyDrive/pill_project'
train_json_dir = os.path.join(DRIVE_BASE, 'train_annotations')
train_image_dir = os.path.join(DRIVE_BASE, 'train_images')

# 경로 확인
print(f"JSON 디렉토리 존재: {os.path.exists(train_json_dir)}")
print(f"이미지 디렉토리 존재: {os.path.exists(train_image_dir)}")

# 데이터 로드 실행
if os.path.exists(train_json_dir) and os.path.exists(train_image_dir):
    df_all = load_all_images_and_annotations(train_json_dir, train_image_dir)

    # 어노테이션이 있는 데이터만 추출
    df = df_all[df_all['has_annotation'] == True].copy()
    print(f"\n어노테이션 있는 데이터: {len(df)}개")

    # 완전성 검사를 위한 추가 처리
    def get_required_count(file_name):
        """파일명에서 기대되는 객체 개수를 계산"""
        prefix_part = file_name.split('_')[0]
        prefixes_raw = prefix_part.split('-')[1:]
        return len(prefixes_raw)

    # DataFrame에 required_count 컬럼 추가
    df['required_count'] = df['file_name'].apply(get_required_count)

    # 이미지별 실제 어노테이션 개수 집계
    actual_counts = df.groupby('file_name').size().reset_index(name='actual_count')
    df = df.merge(actual_counts, on='file_name')

    # 완전한 어노테이션만 필터링
    df_complete = df[df['required_count'] == df['actual_count']].copy()

    print(f"완전한 어노테이션 데이터: {len(df_complete)}개")
    print(f"완전한 어노테이션 이미지: {df_complete['file_name'].nunique()}장")

    # 불완전한 이미지 리스트 생성
    incomplete_df = df[df['required_count'] != df['actual_count']]
    incomplete_image_list = incomplete_df['file_name'].unique().tolist()
    print(f"불완전한 어노테이션 이미지: {len(incomplete_image_list)}장")

else:
    print("필요한 디렉토리가 존재하지 않습니다.")
    print(f"train_annotations: {train_json_dir}")
    print(f"train_images: {train_image_dir}")

JSON 디렉토리 존재: True
이미지 디렉토리 존재: True
총 4526개의 JSON 파일을 찾았습니다.


JSON 파일 처리 중: 100%|██████████| 4526/4526 [25:54<00:00,  2.91it/s]


전체 이미지: 1489장
어노테이션 있는 이미지: 1489장
어노테이션 없는 이미지: 0장


어노테이션 없는 이미지 처리: 0it [00:00, ?it/s]

총 4526개의 데이터를 로드했습니다.
- 어노테이션 있음: 4526개
- 어노테이션 없음: 0개

어노테이션 있는 데이터: 4526개
완전한 어노테이션 데이터: 2406개
완전한 어노테이션 이미지: 639장
불완전한 어노테이션 이미지: 850장


In [ ]:
# 수정된 데이터 로드 함수 - 모든 이미지 포함
def load_all_images_and_annotations(json_dir, image_dir):
    """
    모든 이미지를 로드하고, 어노테이션 유무를 구분
    """
    # 1. JSON 파일에서 어노테이션 데이터 수집
    json_files = glob.glob(os.path.join(json_dir, '**', '*.json'), recursive=True)
    print(f"총 {len(json_files)}개의 JSON 파일을 찾았습니다.")

    annotated_data = []
    image_info_dict = {}  # 이미지 정보 저장용

    for json_file in tqdm(json_files, desc="JSON 파일 처리 중"):
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)

        if not data.get('images'):
            continue

        image_info = data['images'][0]
        image_id = image_info['id']
        file_name = image_info['file_name']
        image_path = os.path.join(image_dir, file_name)

        # 이미지 기본 정보 저장
        image_info_dict[file_name] = {
            'image_id': image_id,
            'image_path': image_path,
            'file_name': file_name,
            'width': image_info['width'],
            'height': image_info['height'],
            'dl_name': image_info['dl_name'],
            'drug_shape': image_info.get('drug_shape'),
            'color_class1': image_info.get('color_class1'),
            'line_front': image_info.get('line_front'),
        }

        # 어노테이션이 있는 경우
        if 'annotations' in data and data['annotations']:
            for anno in data['annotations']:
                if 'bbox' in anno and anno['bbox'] and len(anno['bbox']) == 4:
                    annotated_data.append({
                        **image_info_dict[file_name],  # 이미지 기본 정보
                        'bbox': anno['bbox'],
                        'category_id': anno['category_id'],
                        'area': anno['area'],
                        'has_annotation': True
                    })

    # 2. 전체 이미지 파일 목록 가져오기
    all_image_files = glob.glob(os.path.join(image_dir, '*.png'))
    all_image_names = [os.path.basename(f) for f in all_image_files]

    # 3. 어노테이션이 없는 이미지들 찾기
    annotated_image_names = set(image_info_dict.keys())
    unlabeled_image_names = set(all_image_names) - annotated_image_names

    print(f"전체 이미지: {len(all_image_names)}장")
    print(f"어노테이션 있는 이미지: {len(annotated_image_names)}장")
    print(f"어노테이션 없는 이미지: {len(unlabeled_image_names)}장")

    # 4. 어노테이션 없는 이미지들도 DataFrame에 추가 (bbox 없이)
    unlabeled_data = []
    for img_name in tqdm(unlabeled_image_names, desc="어노테이션 없는 이미지 처리"):
        img_path = os.path.join(image_dir, img_name)
        if os.path.exists(img_path):
            # 이미지 크기 읽기
            img = cv2.imread(img_path)
            height, width = img.shape[:2] if img is not None else (640, 640)

            unlabeled_data.append({
                'image_id': None,  # ID 없음
                'image_path': img_path,
                'file_name': img_name,
                'width': width,
                'height': height,
                'dl_name': None,  # 약품 이름 없음
                'drug_shape': None,
                'color_class1': None,
                'line_front': None,
                'bbox': None,  # bbox 없음
                'category_id': None,
                'area': None,
                'has_annotation': False  # 어노테이션 없음을 표시
            })

    # 5. 모든 데이터 합치기
    all_data = annotated_data + unlabeled_data
    df_all = pd.DataFrame(all_data)

    print(f"총 {len(all_data)}개의 데이터를 로드했습니다.")
    print(f"- 어노테이션 있음: {len(annotated_data)}개")
    print(f"- 어노테이션 없음: {len(unlabeled_data)}개")

    return df_all


In [ ]:
# Drive 경로용 완전한 데이터 처리 코드

import os
import json
import glob
import pandas as pd
import cv2
from tqdm import tqdm

# Drive 경로 설정
DRIVE_BASE = '/content/drive/MyDrive/pill_project'

# 수정된 데이터 로드 함수 - 모든 이미지 포함
def load_all_images_and_annotations(json_dir, image_dir):
    """
    모든 이미지를 로드하고, 어노테이션 유무를 구분
    """
    # 1. JSON 파일에서 어노테이션 데이터 수집
    json_files = glob.glob(os.path.join(json_dir, '**', '*.json'), recursive=True)
    print(f"총 {len(json_files)}개의 JSON 파일을 찾았습니다.")

    annotated_data = []
    image_info_dict = {}  # 이미지 정보 저장용

    for json_file in tqdm(json_files, desc="JSON 파일 처리 중"):
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)

        if not data.get('images'):
            continue

        image_info = data['images'][0]
        image_id = image_info['id']
        file_name = image_info['file_name']
        image_path = os.path.join(image_dir, file_name)

        # 이미지 기본 정보 저장
        image_info_dict[file_name] = {
            'image_id': image_id,
            'image_path': image_path,
            'file_name': file_name,
            'width': image_info['width'],
            'height': image_info['height'],
            'dl_name': image_info['dl_name'],
            'drug_shape': image_info.get('drug_shape'),
            'color_class1': image_info.get('color_class1'),
            'line_front': image_info.get('line_front'),
        }

        # 어노테이션이 있는 경우
        if 'annotations' in data and data['annotations']:
            for anno in data['annotations']:
                if 'bbox' in anno and anno['bbox'] and len(anno['bbox']) == 4:
                    annotated_data.append({
                        **image_info_dict[file_name],  # 이미지 기본 정보
                        'bbox': anno['bbox'],
                        'category_id': anno['category_id'],
                        'area': anno['area'],
                        'has_annotation': True
                    })

    # 2. 전체 이미지 파일 목록 가져오기
    all_image_files = glob.glob(os.path.join(image_dir, '*.png'))
    all_image_names = [os.path.basename(f) for f in all_image_files]

    # 3. 어노테이션이 없는 이미지들 찾기
    annotated_image_names = set(image_info_dict.keys())
    unlabeled_image_names = set(all_image_names) - annotated_image_names

    print(f"전체 이미지: {len(all_image_names)}장")
    print(f"어노테이션 있는 이미지: {len(annotated_image_names)}장")
    print(f"어노테이션 없는 이미지: {len(unlabeled_image_names)}장")

    # 4. 어노테이션 없는 이미지들도 DataFrame에 추가 (bbox 없이)
    unlabeled_data = []
    for img_name in tqdm(unlabeled_image_names, desc="어노테이션 없는 이미지 처리"):
        img_path = os.path.join(image_dir, img_name)
        if os.path.exists(img_path):
            # 이미지 크기 읽기
            img = cv2.imread(img_path)
            height, width = img.shape[:2] if img is not None else (640, 640)

            unlabeled_data.append({
                'image_id': None,  # ID 없음
                'image_path': img_path,
                'file_name': img_name,
                'width': width,
                'height': height,
                'dl_name': None,  # 약품 이름 없음
                'drug_shape': None,
                'color_class1': None,
                'line_front': None,
                'bbox': None,  # bbox 없음
                'category_id': None,
                'area': None,
                'has_annotation': False  # 어노테이션 없음을 표시
            })

    # 5. 모든 데이터 합치기
    all_data = annotated_data + unlabeled_data
    df_all = pd.DataFrame(all_data)

    print(f"총 {len(all_data)}개의 데이터를 로드했습니다.")
    print(f"- 어노테이션 있음: {len(annotated_data)}개")
    print(f"- 어노테이션 없음: {len(unlabeled_data)}개")

    return df_all

# Drive 기반 데이터 로드
train_json_dir = os.path.join(DRIVE_BASE, 'train_annotations')
train_image_dir = os.path.join(DRIVE_BASE, 'train_images')

print(f"JSON 디렉토리: {train_json_dir}")
print(f"이미지 디렉토리: {train_image_dir}")
print(f"JSON 디렉토리 존재: {os.path.exists(train_json_dir)}")
print(f"이미지 디렉토리 존재: {os.path.exists(train_image_dir)}")

# 데이터 로드 실행
df_all = load_all_images_and_annotations(train_json_dir, train_image_dir)

# 어노테이션이 있는 데이터만 추출 (이후 완전성 검사를 위해)
df = df_all[df_all['has_annotation'] == True].copy()

print(f"\n=== 어노테이션 있는 데이터로 완전성 검사 시작 ===")
print(f"어노테이션 데이터: {len(df)}개")

# 어노테이션 누락 데이터 전처리

# 이미지별 required_count 계산 함수
def get_required_count(file_name):
    """
    파일명에서 기대되는 객체 개수를 계산
    예: K-001900-010224-016551-031705_... -> 4개 객체 기대
    """
    prefix_part = file_name.split('_')[0]  # "K-001900-010224-016551-031705"
    prefixes_raw = prefix_part.split('-')[1:]   # ['001900', '010224', '016551', '031705']
    return len(prefixes_raw)

# DataFrame에 required_count 컬럼 추가
print("파일명에서 기대 객체 수 계산 중...")
df['required_count'] = df['file_name'].apply(get_required_count)

# 이미지별 실제 어노테이션 개수 집계
print("이미지별 실제 어노테이션 개수 집계 중...")
actual_counts = df.groupby('file_name').size().reset_index(name='actual_count')

# df와 merge
df = df.merge(actual_counts, on='file_name')

# 완전성 분석
print("\n=== 어노테이션 완전성 분석 ===")
complete_images = df[df['required_count'] == df['actual_count']]['file_name'].nunique()
incomplete_images = df[df['required_count'] != df['actual_count']]['file_name'].nunique()
total_images = df['file_name'].nunique()

print(f"전체 이미지 수: {total_images}장")
print(f"완전한 어노테이션 이미지: {complete_images}장 ({complete_images/total_images*100:.1f}%)")
print(f"불완전한 어노테이션 이미지: {incomplete_images}장 ({incomplete_images/total_images*100:.1f}%)")

# required_count == actual_count인 경우만 필터링
df_complete = df[df['required_count'] == df['actual_count']].copy()

print(f"\n=== 데이터 필터링 결과 ===")
print(f"원본 어노테이션 데이터: {len(df)}개")
print(f"완전한 어노테이션 데이터: {len(df_complete)}개")
print(f"제거된 데이터: {len(df) - len(df_complete)}개")

# 불완전한 데이터 분석
if incomplete_images > 0:
    print("\n=== 불완전한 어노테이션 분석 ===")
    incomplete_analysis = df[df['required_count'] != df['actual_count']].groupby(['file_name', 'required_count', 'actual_count']).size().reset_index(name='count')
    incomplete_summary = incomplete_analysis.groupby(['required_count', 'actual_count']).size().reset_index(name='image_count')

    print("기대 vs 실제 어노테이션 개수 분포:")
    for _, row in incomplete_summary.iterrows():
        print(f"  기대 {row['required_count']}개 -> 실제 {row['actual_count']}개: {row['image_count']}장 이미지")

    # 샘플 출력 (에러 수정)
    print("\n불완전한 어노테이션 예시 (상위 5개):")
    sample_incomplete = df[df['required_count'] != df['actual_count']].groupby('file_name').first().head()
    sample_incomplete = sample_incomplete.reset_index()  # 인덱스 리셋하여 file_name을 컬럼으로 복원
    print(sample_incomplete[['file_name', 'required_count', 'actual_count']])

# 완전한 데이터의 클래스 분포 확인
if len(df_complete) > 0:
    print("\n=== 완전한 어노테이션 데이터의 클래스 분포 ===")
    class_counts_complete = df_complete['dl_name'].value_counts()
    print(f"고유한 약품 클래스 수: {len(class_counts_complete)}")
    print("\n상위 10개 클래스:")
    print(class_counts_complete.head(10))

    # 클래스별 이미지 수도 확인
    print("\n=== 클래스별 이미지 수 ===")
    images_per_class = df_complete.groupby('dl_name')['file_name'].nunique().sort_values(ascending=False)
    print("상위 10개 클래스별 이미지 수:")
    print(images_per_class.head(10))

# 최종 데이터셋 정보
print("\n=== 최종 데이터셋 정보 ===")
print(f"사용 가능한 완전한 어노테이션 데이터: {len(df_complete)}개")
print(f"고유 이미지 수: {df_complete['file_name'].nunique()}장")
print(f"평균 객체 수/이미지: {len(df_complete)/df_complete['file_name'].nunique():.2f}개")

# 샘플 데이터 확인
print("\n=== 완전한 어노테이션 데이터 샘플 ===")
print(df_complete[['file_name', 'dl_name', 'required_count', 'actual_count', 'bbox']].head())

# 불완전한 데이터를 수도 라벨링 대상으로 분리
if incomplete_images > 0:
    df_incomplete = df[df['required_count'] != df['actual_count']].copy()
    incomplete_image_list = df_incomplete['file_name'].unique()

    print(f"\n=== 수도 라벨링 대상 ===")
    print(f"불완전한 어노테이션을 가진 이미지: {len(incomplete_image_list)}장")
    print("이 이미지들을 수도 라벨링으로 보완할 수 있습니다.")
else:
    print("\n모든 이미지가 완전한 어노테이션을 가지고 있습니다.")
    incomplete_image_list = []

# 결과 반환용 변수들 (전역 변수로 저장)
globals()['df_complete'] = df_complete
globals()['incomplete_image_list'] = incomplete_image_list
if incomplete_images > 0:
    globals()['df_incomplete'] = df_incomplete

print(f"\n=== 변수 저장 완료 ===")
print(f"df_complete: {len(df_complete)}개 데이터")
print(f"incomplete_image_list: {len(incomplete_image_list)}장 이미지")
print("Drive 기반 데이터 처리 완료!")

JSON 디렉토리: /content/drive/MyDrive/pill_project/train_annotations
이미지 디렉토리: /content/drive/MyDrive/pill_project/train_images
JSON 디렉토리 존재: True
이미지 디렉토리 존재: True
총 4526개의 JSON 파일을 찾았습니다.


JSON 파일 처리 중: 100%|██████████| 4526/4526 [00:15<00:00, 287.48it/s]


전체 이미지: 1489장
어노테이션 있는 이미지: 1489장
어노테이션 없는 이미지: 0장


어노테이션 없는 이미지 처리: 0it [00:00, ?it/s]


총 4526개의 데이터를 로드했습니다.
- 어노테이션 있음: 4526개
- 어노테이션 없음: 0개

=== 어노테이션 있는 데이터로 완전성 검사 시작 ===
어노테이션 데이터: 4526개
파일명에서 기대 객체 수 계산 중...
이미지별 실제 어노테이션 개수 집계 중...

=== 어노테이션 완전성 분석 ===
전체 이미지 수: 1489장
완전한 어노테이션 이미지: 639장 (42.9%)
불완전한 어노테이션 이미지: 850장 (57.1%)

=== 데이터 필터링 결과 ===
원본 어노테이션 데이터: 4526개
완전한 어노테이션 데이터: 2406개
제거된 데이터: 2120개

=== 불완전한 어노테이션 분석 ===
기대 vs 실제 어노테이션 개수 분포:
  기대 3개 -> 실제 1개: 36장 이미지
  기대 3개 -> 실제 2개: 108장 이미지
  기대 4개 -> 실제 1개: 28장 이미지
  기대 4개 -> 실제 2개: 194장 이미지
  기대 4개 -> 실제 3개: 484장 이미지

불완전한 어노테이션 예시 (상위 5개):
                                           file_name  required_count  \
0  K-001900-010224-016551-031705_0_2_0_2_70_000_2...               4   
1  K-001900-010224-016551-031705_0_2_0_2_75_000_2...               4   
2  K-001900-010224-016551-031705_0_2_0_2_90_000_2...               4   
3  K-001900-010224-016551-033009_0_2_0_2_70_000_2...               4   
4  K-001900-010224-016551-033009_0_2_0_2_75_000_2...               4   

   actual_count  
0             3  
1     

# TTA, 앙상블 해보기


## 데이터 전처리

In [12]:
import os
import yaml
import shutil
from ultralytics import YOLO
from sklearn.model_selection import train_test_split
import glob
from tqdm import tqdm

# Drive 경로 설정
DRIVE_BASE = '/content/drive/MyDrive/pill_project'

# 완전한 어노테이션 데이터만 사용하여 YOLO 데이터셋 생성
def create_clean_yolo_dataset(df_complete, output_dir, class_to_idx):
    """완전한 어노테이션만으로 깨끗한 데이터셋 생성"""
    print(f"완전한 어노테이션 데이터로 데이터셋 생성: {len(df_complete)}개")

    # 디렉토리 생성
    for subset in ['train', 'val']:
        os.makedirs(os.path.join(output_dir, subset, 'images'), exist_ok=True)
        os.makedirs(os.path.join(output_dir, subset, 'labels'), exist_ok=True)

    # 이미지별로 그룹화
    unique_images = df_complete['file_name'].unique()
    train_images, val_images = train_test_split(unique_images, test_size=0.2, random_state=42)

    print(f"Train: {len(train_images)}장, Val: {len(val_images)}장")

    # Train 데이터 처리
    for file_name in tqdm(train_images, desc="Train 데이터 생성"):
        img_annotations = df_complete[df_complete['file_name'] == file_name]

        # 이미지 복사
        src_path = img_annotations.iloc[0]['image_path']
        dst_path = os.path.join(output_dir, 'train', 'images', file_name)
        if not os.path.exists(dst_path):
            shutil.copy2(src_path, dst_path)

        # 라벨 파일 생성
        label_path = os.path.join(output_dir, 'train', 'labels',
                                  os.path.splitext(file_name)[0] + '.txt')
        with open(label_path, 'w') as f:
            for _, row in img_annotations.iterrows():
                class_id = class_to_idx[row['dl_name']]
                bbox = row['bbox']

                # YOLO 형식 변환
                x_center = (bbox[0] + bbox[2] / 2) / row['width']
                y_center = (bbox[1] + bbox[3] / 2) / row['height']
                width = bbox[2] / row['width']
                height = bbox[3] / row['height']

                f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

    # Val 데이터 처리
    for file_name in tqdm(val_images, desc="Val 데이터 생성"):
        img_annotations = df_complete[df_complete['file_name'] == file_name]

        src_path = img_annotations.iloc[0]['image_path']
        dst_path = os.path.join(output_dir, 'val', 'images', file_name)
        if not os.path.exists(dst_path):
            shutil.copy2(src_path, dst_path)

        label_path = os.path.join(output_dir, 'val', 'labels',
                                  os.path.splitext(file_name)[0] + '.txt')
        with open(label_path, 'w') as f:
            for _, row in img_annotations.iterrows():
                class_id = class_to_idx[row['dl_name']]
                bbox = row['bbox']

                x_center = (bbox[0] + bbox[2] / 2) / row['width']
                y_center = (bbox[1] + bbox[3] / 2) / row['height']
                width = bbox[2] / row['width']
                height = bbox[3] / row['height']

                f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

    print("데이터셋 생성 완료")

# 클래스 매핑 (기존 df_complete 사용)
class_names = sorted(df_complete['dl_name'].unique())
class_to_idx = {name: i for i, name in enumerate(class_names)}
idx_to_class = {i: name for i, name in enumerate(class_names)}

# 깨끗한 데이터셋 생성
clean_dataset_dir = os.path.join(DRIVE_BASE, 'pill_dataset_clean')
create_clean_yolo_dataset(df_complete, clean_dataset_dir, class_to_idx)

# YAML 파일 생성
dataset_info = {
    'path': clean_dataset_dir,
    'train': 'train/images',
    'val': 'val/images',
    'names': idx_to_class
}

yaml_path = os.path.join(DRIVE_BASE, 'pill_clean.yaml')
with open(yaml_path, 'w') as f:
    yaml.dump(dataset_info, f)

print(f"YAML 파일 생성: {yaml_path}")

NameError: name 'df_complete' is not defined

## 모델 학습

In [7]:
# 앙상블을 위한 다양한 모델 학습
def train_ensemble_models(yaml_path, output_dir, num_models=3):
    """다양한 설정으로 여러 모델 학습"""
    models_info = []

    print(f"\n=== {num_models}개 앙상블 모델 학습 시작 ===\n")

    # 모델 1: YOLOv8n (기본 설정)
    print("모델 1/3: YOLOv8n 기본 설정")
    model1 = YOLO('yolov8n.pt')
    model1.train(
        data=yaml_path,
        epochs=30,
        batch=16,
        imgsz=640,
        seed=42,
        project=output_dir,
        name='ensemble_model1',
        verbose=True,
        patience=10
    )
    models_info.append(os.path.join(output_dir, 'ensemble_model1/weights/best.pt'))

    # 모델 2: YOLOv8s (더 큰 모델)
    print("\n모델 2/3: YOLOv8s")
    model2 = YOLO('yolov8s.pt')
    model2.train(
        data=yaml_path,
        epochs=30,
        batch=12,
        imgsz=640,
        seed=123,
        project=output_dir,
        name='ensemble_model2',
        verbose=True,
        patience=10
    )
    models_info.append(os.path.join(output_dir, 'ensemble_model2/weights/best.pt'))

    # 모델 3: YOLOv8n (다른 seed + augmentation)
    print("\n모델 3/3: YOLOv8n 강한 증강")
    model3 = YOLO('yolov8n.pt')
    model3.train(
        data=yaml_path,
        epochs=30,
        batch=16,
        imgsz=640,
        seed=456,
        hsv_h=0.02,
        hsv_s=0.7,
        hsv_v=0.4,
        degrees=10,
        translate=0.1,
        scale=0.5,
        flipud=0.5,
        mosaic=1.0,
        project=output_dir,
        name='ensemble_model3',
        verbose=True,
        patience=10
    )
    models_info.append(os.path.join(output_dir, 'ensemble_model3/weights/best.pt'))

    print(f"\n앙상블 모델 학습 완료: {len(models_info)}개")
    return models_info

# 앙상블 모델 학습
ensemble_dir = os.path.join(DRIVE_BASE, 'ensemble_clean')
ensemble_models = train_ensemble_models(yaml_path, ensemble_dir)

NameError: name 'DRIVE_BASE' is not defined

In [5]:
# 변수 정의
yaml_path = '/content/drive/MyDrive/pill_project/pill_dataset.yaml'
ensemble_dir = '/content/drive/MyDrive/pill_project/ensemble_clean'

# 모델 3: CPU로 학습 (설정 축소)
print("모델 3: YOLOv8n 강한 증강 (CPU)")
model3 = YOLO('yolov8n.pt')
model3.train(
    data=yaml_path,
    epochs=15,  # 30 → 15 (시간 단축)
    batch=8,    # 16 → 8 (메모리 절약)
    imgsz=640,
    seed=456,
    device='cpu',  # CPU 명시
    workers=2,     # worker 수 줄이기
    hsv_h=0.02,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=10,
    translate=0.1,
    scale=0.5,
    flipud=0.5,
    mosaic=1.0,
    project=ensemble_dir,
    name='ensemble_model3',
    verbose=True,
    patience=10
)

model3_path = os.path.join(ensemble_dir, 'ensemble_model3/weights/best.pt')
print(f"모델3 학습 완료: {model3_path}")

모델 3: YOLOv8n 강한 증강 (CPU)
Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (AMD EPYC 7B12)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/pill_project/pill_dataset.yaml, degrees=10, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.02, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=ensemble_model34, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True,

/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:582: UserWarning: Glyph 44032 (\N{HANGUL SYLLABLE GA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:582: UserWarning: Glyph 48148 (\N{HANGUL SYLLABLE BA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:582: UserWarning: Glyph 53664 (\N{HANGUL SYLLABLE TO}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:582: UserWarning: Glyph 54028 (\N{HANGUL SYLLABLE PA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:582: UserWarning: Glyph 51221 (\N{HANGUL SYLLABLE JEONG}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
/usr/local/lib/python3.12/dist-p

                   all        151        560      0.648      0.681      0.706      0.635
           가바토파정 100mg         20         20      0.625          1      0.975      0.891
             글리아타민연질캡슐          2          2      0.251      0.502      0.448      0.403
        글리틴정(콜린알포세레이트)          1          1      0.124          1      0.332      0.332
    기넥신에프정(은행엽엑스)(수출용)         40         40      0.965          1      0.995      0.883
         낙소졸정 500/20mg          2          2      0.295          1      0.828      0.745
             노바스크정 5mg         10         10      0.787          1      0.995      0.879
              놀텍정 10mg          8          8          1      0.462      0.811      0.772
          뉴로메드정(옥시라세탐)          4          4      0.189      0.645       0.21      0.189
         다보타민큐정 10mg/병          6          6      0.657      0.167       0.57      0.506
         동아가바펜틴정 800mg         23         23      0.546          1      0.959      0.893
            라비에트정 20m

## 모델들로 앙상블.

In [7]:
!pip install ensemble-boxes ultralytics

from google.colab import drive
import os
import numpy as np
from ensemble_boxes import weighted_boxes_fusion
from ultralytics import YOLO

drive.mount('/content/drive')

# 3개 모델 모두 사용
ensemble_models = [
    '/content/drive/MyDrive/pill_project/ensemble_clean/ensemble_model1/weights/best.pt',
    '/content/drive/MyDrive/pill_project/ensemble_clean/ensemble_model2/weights/best.pt',
    '/content/drive/MyDrive/pill_project/ensemble_clean/ensemble_model34/weights/best.pt'
]

# 모델 존재 확인
print("=== 모델 확인 ===")
for i, model_path in enumerate(ensemble_models):
    exists = os.path.exists(model_path)
    print(f"{i+1}. {os.path.basename(os.path.dirname(os.path.dirname(model_path)))}: {exists}")


def ensemble_predict_wbf(models, image_path, iou_thr=0.5, skip_box_thr=0.25, weights=None):
    """여러 모델의 예측을 WBF로 결합"""
    all_boxes = []
    all_scores = []
    all_labels = []

    # 각 모델로 예측
    for model_path in models:
        model = YOLO(model_path)
        results = model.predict(
            image_path,
            conf=0.001,  # 낮은 threshold
            verbose=False,
            augment=False  # TTA 끄기 (효과 없었음)
        )

        boxes_list = []
        scores_list = []
        labels_list = []

        for result in results:
            if result.boxes is not None and len(result.boxes) > 0:
                boxes = result.boxes.xyxyn.cpu().numpy()  # normalized
                scores = result.boxes.conf.cpu().numpy()
                labels = result.boxes.cls.cpu().numpy().astype(int)

                boxes_list.append(boxes)
                scores_list.append(scores)
                labels_list.append(labels)

        if boxes_list:
            all_boxes.append(np.vstack(boxes_list))
            all_scores.append(np.concatenate(scores_list))
            all_labels.append(np.concatenate(labels_list))
        else:
            # 빈 배열 추가
            all_boxes.append(np.array([]).reshape(0, 4))
            all_scores.append(np.array([]))
            all_labels.append(np.array([]))

    # WBF 적용
    if weights is None:
        weights = [1.0] * len(models)

    boxes, scores, labels = weighted_boxes_fusion(
        all_boxes,
        all_scores,
        all_labels,
        weights=weights,
        iou_thr=iou_thr,
        skip_box_thr=skip_box_thr
    )

    return boxes, scores, labels

# 테스트 이미지로 확인
test_image = '/content/drive/MyDrive/pill_project/test_images/1.png'

print(f"\n=== 테스트 예측 ===")
print(f"이미지: {test_image}")

if os.path.exists(test_image):
    boxes, scores, labels = ensemble_predict_wbf(ensemble_models, test_image)
    print(f"앙상블 결과: {len(boxes)}개 객체 검출")
    print(f"평균 confidence: {scores.mean():.3f}")
else:
    print("테스트 이미지가 없습니다. 경로를 확인하세요.")
    # 다른 이미지 시도
    import glob
    test_images = glob.glob('/content/drive/MyDrive/pill_project/test_images/*.png')[:1]
    if test_images:
        test_image = test_images[0]
        print(f"대체 이미지 사용: {test_image}")
        boxes, scores, labels = ensemble_predict_wbf(ensemble_models, test_image)
        print(f"앙상블 결과: {len(boxes)}개 객체 검출")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
=== 모델 확인 ===
1. ensemble_model1: True
2. ensemble_model2: True
3. ensemble_model34: True

=== 테스트 예측 ===
이미지: /content/drive/MyDrive/pill_project/test_images/1.png
앙상블 결과: 5개 객체 검출
평균 confidence: 0.625


## 모델 3개를 앙상블로 했음 <font size = '3'> ( 모델 3개 :  </font>

In [10]:
# 필요한 라이브러리
import pandas as pd
import json
import glob
import os
import numpy as np
from tqdm import tqdm
from PIL import Image
from ensemble_boxes import weighted_boxes_fusion
from ultralytics import YOLO

# 모델 경로
ensemble_models = [
    '/content/drive/MyDrive/pill_project/ensemble_clean/ensemble_model1/weights/best.pt',
    '/content/drive/MyDrive/pill_project/ensemble_clean/ensemble_model2/weights/best.pt',
    '/content/drive/MyDrive/pill_project/ensemble_clean/ensemble_model34/weights/best.pt'  # 추가
]
# WBF 앙상블 함수
def ensemble_predict_wbf(models, image_path, iou_thr=0.5, skip_box_thr=0.25, weights=None):
    """여러 모델의 예측을 WBF로 결합"""
    all_boxes = []
    all_scores = []
    all_labels = []

    for model_path in models:
        model = YOLO(model_path)
        results = model.predict(image_path, conf=0.001, verbose=False, augment=False)

        boxes_list = []
        scores_list = []
        labels_list = []

        for result in results:
            if result.boxes is not None and len(result.boxes) > 0:
                boxes = result.boxes.xyxyn.cpu().numpy()
                scores = result.boxes.conf.cpu().numpy()
                labels = result.boxes.cls.cpu().numpy().astype(int)

                boxes_list.append(boxes)
                scores_list.append(scores)
                labels_list.append(labels)

        if boxes_list:
            all_boxes.append(np.vstack(boxes_list))
            all_scores.append(np.concatenate(scores_list))
            all_labels.append(np.concatenate(labels_list))
        else:
            all_boxes.append(np.array([]).reshape(0, 4))
            all_scores.append(np.array([]))
            all_labels.append(np.array([]))

    if weights is None:
        weights = [1.0] * len(models)

    boxes, scores, labels = weighted_boxes_fusion(
        all_boxes, all_scores, all_labels,
        weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr
    )

    return boxes, scores, labels

# 제출 파일 생성 함수
def create_submission_with_ensemble(models, test_dir, mapping_path, output_csv='submission_ensemble1.csv'):
    with open(mapping_path, 'r') as f:
        yolo_idx_to_cat_id = json.load(f)
        yolo_idx_to_cat_id = {int(k): int(v) for k, v in yolo_idx_to_cat_id.items()}

    test_images = sorted(glob.glob(os.path.join(test_dir, '*.png')))
    print(f"테스트 이미지: {len(test_images)}장")

    predictions = []
    annotation_id = 1

    for img_path in tqdm(test_images, desc="앙상블 추론 중"):
        image_id = int(os.path.splitext(os.path.basename(img_path))[0])

        boxes, scores, labels = ensemble_predict_wbf(models, img_path, iou_thr=0.5, skip_box_thr=0.25)

        img = Image.open(img_path)
        img_width, img_height = img.size

        for box, score, label in zip(boxes, scores, labels):
            x1 = box[0] * img_width
            y1 = box[1] * img_height
            x2 = box[2] * img_width
            y2 = box[3] * img_height

            category_id = yolo_idx_to_cat_id.get(int(label), int(label))

            predictions.append([
                annotation_id, image_id, category_id,
                x1, y1, x2-x1, y2-y1, score
            ])
            annotation_id += 1

    df = pd.DataFrame(predictions, columns=[
        'annotation_id', 'image_id', 'category_id',
        'bbox_x', 'bbox_y', 'bbox_w', 'bbox_h', 'score'
    ])

    df.to_csv(output_csv, index=False)
    print(f"제출 파일 생성: {output_csv}")
    print(f"총 예측 수: {len(predictions)}")

    return df

# 실행
test_dir = '/content/drive/MyDrive/pill_project/test_images'
mapping_path = '/content/drive/MyDrive/pill_project/class_mapping.json'

df_submission = create_submission_with_ensemble(ensemble_models, test_dir, mapping_path)

from google.colab import files
files.download('submission_ensemble1.csv')

테스트 이미지: 843장


앙상블 추론 중: 100%|██████████| 843/843 [10:41<00:00,  1.32it/s]

제출 파일 생성: submission_ensemble1.csv
총 예측 수: 4229


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 단일 모델에 TTA만 적용한 것 <font size = '3'>( 단일모델 :  YOLOv8n ) </font>

-> 역시 성능이 좋지 않았음.

In [5]:
# 단일 모델 + TTA
best_model = YOLO(ensemble_models[1])  # model2 사용

def create_submission_with_tta(model, test_dir, mapping_path, output_csv='submission_tta.csv'):
    """단일 모델 + TTA로 제출 파일 생성"""

    # 클래스 매핑 로드 (전체 경로 사용)
    with open(mapping_path, 'r') as f:
        yolo_idx_to_cat_id = json.load(f)
        yolo_idx_to_cat_id = {int(k): int(v) for k, v in yolo_idx_to_cat_id.items()}

    test_images = sorted(glob.glob(os.path.join(test_dir, '*.png')))
    print(f"테스트 이미지: {len(test_images)}장")

    predictions = []
    annotation_id = 1

    for img_path in tqdm(test_images, desc="TTA 추론 중"):
        image_id = int(os.path.splitext(os.path.basename(img_path))[0])

        # TTA 활성화
        preds = model.predict(
            img_path,
            conf=0.25,
            iou=0.45,
            augment=True,  # TTA
            verbose=False
        )

        # 이미지 크기
        img = Image.open(img_path)
        img_width, img_height = img.size

        for result in preds:
            if result.boxes is not None:
                for box in result.boxes:
                    x1, y1, x2, y2 = box.xyxyn[0].tolist()
                    score = box.conf[0].item()
                    label = int(box.cls[0].item())

                    # pixel 좌표로 변환
                    x1_px = x1 * img_width
                    y1_px = y1 * img_height
                    x2_px = x2 * img_width
                    y2_px = y2 * img_height

                    category_id = yolo_idx_to_cat_id.get(label, label)

                    predictions.append([
                        annotation_id, image_id, category_id,
                        x1_px, y1_px, x2_px-x1_px, y2_px-y1_px, score
                    ])
                    annotation_id += 1

    df = pd.DataFrame(predictions, columns=[
        'annotation_id', 'image_id', 'category_id',
        'bbox_x', 'bbox_y', 'bbox_w', 'bbox_h', 'score'
    ])

    df.to_csv(output_csv, index=False)
    print(f"TTA 제출 파일 생성: {output_csv}")
    print(f"총 예측 수: {len(predictions)}")

    return df

# 실행
mapping_path = '/content/drive/MyDrive/pill_project/class_mapping.json'
df_tta = create_submission_with_tta(best_model, test_dir, mapping_path)

from google.colab import files
files.download('submission_tta.csv')

테스트 이미지: 843장


TTA 추론 중: 100%|██████████| 843/843 [18:01<00:00,  1.28s/it]

TTA 제출 파일 생성: submission_tta.csv
총 예측 수: 3329


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 수도 라벨링

In [ ]:
# Google Drive 마운트 및 앙상블 기반 수도 라벨링 구현

# 1. Google Drive 마운트
from google.colab import drive
import os
drive.mount('/content/drive')

# Drive 내 작업 디렉토리 설정 (필요시 경로 수정)
DRIVE_BASE = '/content/drive/MyDrive/pill_project'  # 본인의 Drive 폴더로 변경
os.makedirs(DRIVE_BASE, exist_ok=True)
os.chdir(DRIVE_BASE)

import numpy as np
from collections import defaultdict
import shutil
import yaml
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from ultralytics import YOLO
import cv2
import glob
import json

print(f"작업 디렉토리: {os.getcwd()}")

# 2. 데이터 경로 확인 (Drive 내) - 수정된 경로
train_json_dir = os.path.join(DRIVE_BASE, 'train_annotations')  # data 제거
train_image_dir = os.path.join(DRIVE_BASE, 'train_images')      # data 제거

print(f"JSON 디렉토리 존재: {os.path.exists(train_json_dir)}")
print(f"이미지 디렉토리 존재: {os.path.exists(train_image_dir)}")

# df_complete가 없으면 로드 (Drive에서)
if 'df_complete' not in globals():
    print("df_complete 로드 중...")
    # 간단한 로드 함수
    def quick_load_complete_data():
        json_files = glob.glob(os.path.join(train_json_dir, '**', '*.json'), recursive=True)
        data = []

        for json_file in tqdm(json_files, desc="JSON 로딩 중"):
            with open(json_file, 'r', encoding='utf-8') as f:
                json_data = json.load(f)

            if not json_data.get('images') or not json_data.get('annotations'):
                continue

            image_info = json_data['images'][0]

            # 기대 객체 수 계산
            prefix_part = image_info['file_name'].split('_')[0]
            required_count = len(prefix_part.split('-')[1:])

            # 실제 어노테이션 수
            actual_count = len(json_data['annotations'])

            # 완전한 어노테이션만 선택
            if required_count == actual_count:
                for anno in json_data['annotations']:
                    if 'bbox' in anno and anno['bbox'] and len(anno['bbox']) == 4:
                        data.append({
                            'file_name': image_info['file_name'],
                            'image_path': os.path.join(train_image_dir, image_info['file_name']),
                            'width': image_info['width'],
                            'height': image_info['height'],
                            'dl_name': image_info['dl_name'],
                            'category_id': anno['category_id'],
                            'bbox': anno['bbox'],
                            'has_annotation': True
                        })

        return pd.DataFrame(data)

    df_complete = quick_load_complete_data()
    print(f"df_complete 로드 완료: {len(df_complete)}개")

# 불완전한 이미지 리스트도 로드 (필요시)
if 'incomplete_image_list' not in globals():
    # 전체 이미지에서 완전한 이미지 제외
    all_images = set([os.path.basename(f) for f in glob.glob(os.path.join(train_image_dir, '*.png'))])
    complete_images = set(df_complete['file_name'].unique())
    incomplete_image_list = list(all_images - complete_images)
    print(f"불완전한 이미지: {len(incomplete_image_list)}장")

# 3. 클래스 매핑 생성 (완전한 데이터 기준)
class_names = sorted(df_complete['dl_name'].unique())
name_to_cat_id_map = df_complete.drop_duplicates('dl_name').set_index('dl_name')['category_id'].to_dict()
idx_to_class = {i: name for i, name in enumerate(class_names)}
yolo_idx_to_original_cat_id = {i: name_to_cat_id_map[name] for i, name in idx_to_class.items()}
class_to_idx = {name: i for i, name in enumerate(class_names)}

print(f"클래스 매핑 완료: {len(class_names)}개 클래스")

# 4. YAML 파일 생성 (Drive 내)
dataset_info = {
    'path': os.path.join(DRIVE_BASE, 'pill_dataset'),
    'train': 'train/images',
    'val': 'val/images',
    'names': idx_to_class
}

yaml_path = os.path.join(DRIVE_BASE, 'pill_dataset.yaml')
with open(yaml_path, 'w') as f:
    yaml.dump(dataset_info, f)

print(f"YAML 파일 생성: {yaml_path}")

# 5. 완전한 데이터로 YOLO 데이터셋 생성 (Drive 내)
def create_yolo_dataset_from_complete(df_complete, output_dir, class_to_idx):
    """완전한 어노테이션 데이터로 YOLO 데이터셋 생성"""
    print(f"YOLO 데이터셋 생성 중: {output_dir}")

    # 디렉토리 생성
    os.makedirs(os.path.join(output_dir, 'train', 'images'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'train', 'labels'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'val', 'images'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'val', 'labels'), exist_ok=True)

    # 이미지별로 그룹화
    unique_images = df_complete['file_name'].unique()
    train_images, val_images = train_test_split(unique_images, test_size=0.2, random_state=42)

    train_set = set(train_images)
    val_set = set(val_images)

    # 이미지별로 처리
    for file_name in tqdm(unique_images, desc="YOLO 데이터셋 생성"):
        # 해당 이미지의 모든 어노테이션 가져오기
        img_annotations = df_complete[df_complete['file_name'] == file_name]

        # train/val 결정
        subset = 'train' if file_name in train_set else 'val'

        # 이미지 복사
        src_path = img_annotations.iloc[0]['image_path']
        dst_path = os.path.join(output_dir, subset, 'images', file_name)
        if not os.path.exists(dst_path):
            shutil.copy2(src_path, dst_path)

        # 라벨 파일 생성
        label_path = os.path.join(output_dir, subset, 'labels', os.path.splitext(file_name)[0] + '.txt')

        with open(label_path, 'w') as f:
            for _, row in img_annotations.iterrows():
                class_id = class_to_idx[row['dl_name']]
                bbox = row['bbox']

                # YOLO 형식으로 변환
                x_center = (bbox[0] + bbox[2] / 2) / row['width']
                y_center = (bbox[1] + bbox[3] / 2) / row['height']
                width = bbox[2] / row['width']
                height = bbox[3] / row['height']

                f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

    print(f"학습 이미지: {len(train_images)}장, 검증 이미지: {len(val_images)}장")

# YOLO 데이터셋 생성 (Drive 내)
yolo_dataset_dir = os.path.join(DRIVE_BASE, 'pill_dataset')
create_yolo_dataset_from_complete(df_complete, yolo_dataset_dir, class_to_idx)

# 6. 앙상블 모델 학습 (Drive에 저장) - 절대 경로 수정
def train_ensemble_models(yaml_path, base_dir, num_models=3):
    """서로 다른 설정으로 여러 모델 학습"""
    models = []

    print("=== 앙상블 모델 학습 시작 ===")

    ensemble_dir = os.path.join(base_dir, 'ensemble_models')

    # 모델 1: YOLOv8n (기본)
    print("모델 1: YOLOv8n 기본 설정 학습 중...")
    model1 = YOLO('yolov8n.pt')
    model1.train(
        data=yaml_path,
        epochs=15,
        batch=16,
        imgsz=640,
        project=ensemble_dir,
        name='model1_basic',
        verbose=False
    )
    # 절대 경로로 수정
    model1_path = os.path.join(ensemble_dir, 'model1_basic', 'weights', 'best.pt')
    models.append((model1_path, 'basic'))

    # 모델 2: YOLOv8s (더 큰 모델)
    print("모델 2: YOLOv8s 학습 중...")
    model2 = YOLO('yolov8s.pt')
    model2.train(
        data=yaml_path,
        epochs=15,
        batch=12,
        imgsz=640,
        project=ensemble_dir,
        name='model2_large',
        verbose=False
    )
    model2_path = os.path.join(ensemble_dir, 'model2_large', 'weights', 'best.pt')
    models.append((model2_path, 'large'))

    # 모델 3: YOLOv8n (강한 증강)
    print("모델 3: YOLOv8n 강한 증강 학습 중...")
    model3 = YOLO('yolov8n.pt')
    model3.train(
        data=yaml_path,
        epochs=15,
        batch=16,
        imgsz=640,
        hsv_h=0.02,
        hsv_s=0.8,
        hsv_v=0.5,
        degrees=15,
        translate=0.15,
        scale=0.7,
        flipud=0.7,
        mosaic=1.0,
        mixup=0.3,
        project=ensemble_dir,
        name='model3_augmented',
        verbose=False
    )
    model3_path = os.path.join(ensemble_dir, 'model3_augmented', 'weights', 'best.pt')
    models.append((model3_path, 'augmented'))

    return models

# 앙상블 모델 학습
ensemble_model_paths = train_ensemble_models(yaml_path, DRIVE_BASE)

# 7. IoU 계산 함수
def calculate_iou(box1, box2):
    """두 박스 간 IoU 계산 [x1, y1, x2, y2] 형식"""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    if x2 <= x1 or y2 <= y1:
        return 0.0

    intersection = (x2 - x1) * (y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area1 + area2 - intersection

    return intersection / union if union > 0 else 0.0

# 8. 앙상블 합의 함수
def find_ensemble_agreement(predictions_by_models, iou_threshold=0.5, agreement_threshold=2):
    """여러 모델의 예측에서 합의된 예측만 추출"""
    if len(predictions_by_models) < agreement_threshold:
        return []

    # 모든 예측을 하나로 합치기
    all_predictions = []
    for model_idx, predictions in enumerate(predictions_by_models):
        for pred in predictions:
            pred['model_id'] = model_idx
            all_predictions.append(pred)

    if not all_predictions:
        return []

    agreed_predictions = []
    used_indices = set()

    # 각 예측에 대해 다른 모델들과의 합의 확인
    for i, pred1 in enumerate(all_predictions):
        if i in used_indices:
            continue

        agreements = [pred1]
        agreement_models = {pred1['model_id']}
        current_used = {i}

        # 다른 예측들과 비교
        for j, pred2 in enumerate(all_predictions[i+1:], i+1):
            if j in used_indices or pred2['model_id'] in agreement_models:
                continue

            # IoU와 클래스 체크
            iou = calculate_iou(pred1['bbox'], pred2['bbox'])
            same_class = pred1['class'] == pred2['class']

            if iou >= iou_threshold and same_class:
                agreements.append(pred2)
                agreement_models.add(pred2['model_id'])
                current_used.add(j)

        # 충분한 합의가 있는지 확인
        if len(agreement_models) >= agreement_threshold:
            # 평균 예측 계산
            avg_bbox = [0, 0, 0, 0]
            avg_conf = 0

            for pred in agreements:
                for k in range(4):
                    avg_bbox[k] += pred['bbox'][k]
                avg_conf += pred['conf']

            num_agreements = len(agreements)
            avg_bbox = [coord / num_agreements for coord in avg_bbox]
            avg_conf = avg_conf / num_agreements

            agreed_predictions.append({
                'bbox': avg_bbox,
                'class': pred1['class'],
                'conf': avg_conf,
                'agreement_count': len(agreement_models)
            })

            used_indices.update(current_used)

    return agreed_predictions

# 9. 수도 라벨 생성
def generate_pseudo_labels_for_incomplete(incomplete_image_list, ensemble_model_paths, train_image_dir, confidence_threshold=0.6):
    """불완전한 이미지들에 대해 수도 라벨 생성"""

    # 모델들 로드
    models = []
    for model_path, model_type in ensemble_model_paths:
        print(f"모델 로드 중: {model_path}")
        if os.path.exists(model_path):
            models.append(YOLO(model_path))
        else:
            print(f"모델 파일이 없습니다: {model_path}")

    if not models:
        print("로드된 모델이 없습니다!")
        return []

    pseudo_labels = []

    print(f"불완전한 {len(incomplete_image_list)}장 이미지에 수도 라벨링 시작...")

    for img_name in tqdm(incomplete_image_list, desc="수도 라벨링 중"):
        img_path = os.path.join(train_image_dir, img_name)

        if not os.path.exists(img_path):
            continue

        # 각 모델로 예측
        predictions_by_models = []

        for model in models:
            results = model.predict(img_path, conf=confidence_threshold, verbose=False)
            model_predictions = []

            for result in results:
                if result.boxes is not None:
                    boxes = result.boxes
                    for box in boxes:
                        x1, y1, x2, y2 = box.xyxy[0].tolist()
                        model_predictions.append({
                            'bbox': [x1, y1, x2, y2],
                            'class': int(box.cls[0]),
                            'conf': box.conf[0].item()
                        })

            predictions_by_models.append(model_predictions)

        # 모델들 간 합의된 예측만 선택
        agreed_predictions = find_ensemble_agreement(
            predictions_by_models,
            iou_threshold=0.5,
            agreement_threshold=2
        )

        # 수도 라벨로 저장
        for pred in agreed_predictions:
            pseudo_labels.append({
                'file_name': img_name,
                'image_path': img_path,
                'bbox': pred['bbox'],
                'class': pred['class'],
                'confidence': pred['conf'],
                'agreement_count': pred['agreement_count']
            })

    print(f"총 {len(pseudo_labels)}개의 수도 라벨이 생성되었습니다.")
    return pseudo_labels

# 수도 라벨 생성 실행
print(f"\n=== 수도 라벨링 시작 ===")
print(f"대상 이미지: {len(incomplete_image_list)}장")

pseudo_labels = generate_pseudo_labels_for_incomplete(
    incomplete_image_list,
    ensemble_model_paths,
    train_image_dir,
    confidence_threshold=0.7
)

# 10. 결과 분석
print(f"\n=== 수도 라벨링 결과 ===")
print(f"수도 라벨 생성된 이미지: {len(set([pl['file_name'] for pl in pseudo_labels]))}장")
print(f"총 수도 라벨 개수: {len(pseudo_labels)}개")

if pseudo_labels:
    # 신뢰도별 분포
    confidences = [pl['confidence'] for pl in pseudo_labels]
    print(f"수도 라벨 신뢰도 - 평균: {np.mean(confidences):.3f}, 최소: {np.min(confidences):.3f}, 최대: {np.max(confidences):.3f}")

    # 합의 수준별 분포
    agreement_counts = [pl['agreement_count'] for pl in pseudo_labels]
    for agreement in sorted(set(agreement_counts)):
        count = agreement_counts.count(agreement)
        print(f"  {agreement}개 모델 합의: {count}개 라벨")

# 11. 수도 라벨을 YOLO 형식으로 저장 (Drive 내)
def save_pseudo_labels_yolo(pseudo_labels, output_dir):
    """수도 라벨을 YOLO 형식으로 저장"""
    pseudo_dir = os.path.join(output_dir, 'pseudo')
    os.makedirs(os.path.join(pseudo_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(pseudo_dir, 'labels'), exist_ok=True)

    # 이미지별로 그룹화
    labels_by_image = defaultdict(list)
    for label in pseudo_labels:
        labels_by_image[label['file_name']].append(label)

    for img_name, img_labels in labels_by_image.items():
        # 이미지 복사
        src_path = img_labels[0]['image_path']
        dst_path = os.path.join(pseudo_dir, 'images', img_name)
        if not os.path.exists(dst_path):
            shutil.copy2(src_path, dst_path)

        # 라벨 파일 생성
        label_path = os.path.join(pseudo_dir, 'labels', os.path.splitext(img_name)[0] + '.txt')

        # 이미지 크기 가져오기
        img = cv2.imread(src_path)
        img_height, img_width = img.shape[:2]

        with open(label_path, 'w') as f:
            for label in img_labels:
                bbox = label['bbox']  # [x1, y1, x2, y2]
                class_id = label['class']

                # YOLO 형식으로 변환
                x_center = (bbox[0] + bbox[2]) / 2 / img_width
                y_center = (bbox[1] + bbox[3]) / 2 / img_height
                width = (bbox[2] - bbox[0]) / img_width
                height = (bbox[3] - bbox[1]) / img_height

                f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

    print(f"수도 라벨이 {pseudo_dir}에 저장되었습니다.")

if pseudo_labels:
    save_pseudo_labels_yolo(pseudo_labels, yolo_dataset_dir)

print(f"\n=== 최종 결과 ===")
print(f"완전한 어노테이션 데이터: {len(df_complete)}개")
print(f"생성된 수도 라벨: {len(pseudo_labels)}개")
print(f"전체 사용 가능한 데이터: {len(df_complete) + len(pseudo_labels)}개")
print(f"데이터 증가율: {len(pseudo_labels)/len(df_complete)*100:.1f}%")
print(f"모든 결과가 Drive에 저장되었습니다: {DRIVE_BASE}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
작업 디렉토리: /content/drive/MyDrive/pill_project
JSON 디렉토리 존재: True
이미지 디렉토리 존재: True
클래스 매핑 완료: 73개 클래스
YAML 파일 생성: /content/drive/MyDrive/pill_project/pill_dataset.yaml
YOLO 데이터셋 생성 중: /content/drive/MyDrive/pill_project/pill_dataset


YOLO 데이터셋 생성: 100%|██████████| 639/639 [03:40<00:00,  2.90it/s]


학습 이미지: 511장, 검증 이미지: 128장
=== 앙상블 모델 학습 시작 ===
모델 1: YOLOv8n 기본 설정 학습 중...
Ultralytics 8.3.202 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/pill_project/pill_dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=model1_basic, nbs=64, nms=False, op

수도 라벨링 중: 100%|██████████| 850/850 [17:43<00:00,  1.25s/it]


총 2162개의 수도 라벨이 생성되었습니다.

=== 수도 라벨링 결과 ===
수도 라벨 생성된 이미지: 849장
총 수도 라벨 개수: 2162개
수도 라벨 신뢰도 - 평균: 0.943, 최소: 0.740, 최대: 0.995
  2개 모델 합의: 979개 라벨
  3개 모델 합의: 1183개 라벨
수도 라벨이 /content/drive/MyDrive/pill_project/pill_dataset/pseudo에 저장되었습니다.

=== 최종 결과 ===
완전한 어노테이션 데이터: 2406개
생성된 수도 라벨: 2162개
전체 사용 가능한 데이터: 4568개
데이터 증가율: 89.9%
모든 결과가 Drive에 저장되었습니다: /content/drive/MyDrive/pill_project


## 수도 라벨링한 데이터로 학습

원래는 데이터가 4,500개 가량 되었는데,

용량이 조금 문제가 되어서 원래 데이터 에서 700개 줄이고

수도 라벨링으로 생성한 데이터에서 700개 줄여서 총 1,700개를 줄인 2,800개의 데이터로 학습함.

In [ ]:
# 올바른 데이터 축소: 700개씩 줄여서 더 많은 데이터로 학습

import os
import shutil
import glob
import yaml
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from ultralytics import YOLO
import gc
import torch

def clear_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

print("=== 올바른 데이터 축소 (700개씩 줄이기) ===")
clear_memory()

# 1. df_complete에서 700개 줄이기 (2,406 → 1,706개)
def reduce_original_by_amount(df_complete, reduce_count=700):
    """완전한 어노테이션 데이터에서 지정한 개수만큼 줄이기"""
    print(f"원본 완전한 어노테이션 데이터: {len(df_complete)}개")

    # 이미지별로 그룹화
    unique_images = df_complete['file_name'].unique()
    print(f"원본 이미지 수: {len(unique_images)}장")

    # 평균 객체 수 계산
    avg_objects_per_image = len(df_complete) / len(unique_images)
    print(f"이미지당 평균 객체 수: {avg_objects_per_image:.2f}개")

    # 줄일 이미지 수 계산
    images_to_remove = int(reduce_count / avg_objects_per_image)
    print(f"제거할 이미지 수: {images_to_remove}장")

    # 랜덤하게 이미지 선택해서 제거
    random.seed(42)
    images_to_keep = random.sample(list(unique_images), len(unique_images) - images_to_remove)

    # 선택된 이미지의 모든 어노테이션 유지
    df_reduced = df_complete[df_complete['file_name'].isin(images_to_keep)].copy()

    print(f"축소 후 데이터: {len(df_reduced)}개 ({len(images_to_keep)}장 이미지)")
    print(f"실제 줄어든 양: {len(df_complete) - len(df_reduced)}개")
    return df_reduced

# 2. 수도 라벨에서 700개 줄이기 (2,145 → 1,445개)
def reduce_pseudo_by_amount(pseudo_reduced_dir, target_final_count=1445):
    """축소된 수도 라벨에서 더 많이 선택 (현재보다 늘리기)"""

    if not os.path.exists(pseudo_reduced_dir):
        print(f"축소된 수도 라벨 디렉토리가 없습니다: {pseudo_reduced_dir}")
        return []

    # 현재 축소된 데이터 확인
    pseudo_images = glob.glob(os.path.join(pseudo_reduced_dir, 'images', '*'))
    print(f"현재 축소된 수도 라벨 이미지: {len(pseudo_images)}장")

    # 각 이미지별 라벨 수 계산
    image_label_counts = {}
    total_labels = 0

    for img_path in pseudo_images:
        img_name = os.path.basename(img_path)
        label_name = os.path.splitext(img_name)[0] + '.txt'
        label_path = os.path.join(pseudo_reduced_dir, 'labels', label_name)

        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label_count = len([line for line in f if line.strip()])
            image_label_counts[img_name] = label_count
            total_labels += label_count

    print(f"현재 총 수도 라벨 수: {total_labels}개")

    if total_labels < target_final_count:
        print(f"현재 데이터({total_labels}개)가 목표({target_final_count}개)보다 적습니다.")
        print("모든 데이터를 사용합니다.")
        return list(image_label_counts.keys())

    # 랜덤하게 이미지 선택해서 목표 개수에 맞추기
    selected_images = []
    selected_labels = 0

    image_list = list(image_label_counts.keys())
    random.shuffle(image_list)

    for img_name in image_list:
        if selected_labels + image_label_counts[img_name] <= target_final_count:
            selected_images.append(img_name)
            selected_labels += image_label_counts[img_name]
        elif selected_labels < target_final_count:
            selected_images.append(img_name)
            selected_labels += image_label_counts[img_name]
            break

    print(f"선택된 수도 라벨: {len(selected_images)}장 이미지 ({selected_labels}개 라벨)")
    return selected_images

# 실행
random.seed(42)

# df_complete 확인 및 축소
if 'df_complete' not in globals():
    print("df_complete가 없습니다. 세션 복원 코드를 먼저 실행하세요.")
    exit()

print(f"현재 df_complete: {len(df_complete)}개")
df_larger = reduce_original_by_amount(df_complete, 700)

# 클래스 매핑 생성
class_names = sorted(df_larger['dl_name'].unique())
class_to_idx = {name: i for i, name in enumerate(class_names)}
idx_to_class = {i: name for i, name in enumerate(class_names)}
print(f"클래스 수: {len(class_names)}개")

# 수도 라벨 선택
pseudo_reduced_dir = './pill_dataset/pseudo_reduced'
selected_pseudo_images = reduce_pseudo_by_amount(pseudo_reduced_dir, 1445)

# 3. 더 큰 YOLO 데이터셋 생성
def create_larger_yolo_dataset(df_larger, selected_pseudo_images, pseudo_reduced_dir, output_dir, class_to_idx):
    """더 많은 데이터로 YOLO 데이터셋 생성"""

    print("더 큰 YOLO 데이터셋 생성 중...")

    # 디렉토리 생성
    os.makedirs(os.path.join(output_dir, 'train', 'images'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'train', 'labels'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'val', 'images'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'val', 'labels'), exist_ok=True)

    # 1) 완전한 어노테이션 데이터 처리
    original_images = df_larger['file_name'].unique()
    orig_train, orig_val = train_test_split(original_images, test_size=0.2, random_state=42)

    print(f"원본 데이터 분할: Train {len(orig_train)}장, Val {len(orig_val)}장")

    # 원본 train 데이터
    for file_name in tqdm(orig_train, desc="원본 Train 데이터"):
        img_annotations = df_larger[df_larger['file_name'] == file_name]

        # 이미지 복사
        src_path = img_annotations.iloc[0]['image_path']
        dst_path = os.path.join(output_dir, 'train', 'images', file_name)
        if not os.path.exists(dst_path):
            shutil.copy2(src_path, dst_path)

        # 라벨 파일 생성
        label_path = os.path.join(output_dir, 'train', 'labels', os.path.splitext(file_name)[0] + '.txt')
        with open(label_path, 'w') as f:
            for _, row in img_annotations.iterrows():
                class_id = class_to_idx[row['dl_name']]
                bbox = row['bbox']

                # YOLO 형식으로 변환
                x_center = (bbox[0] + bbox[2] / 2) / row['width']
                y_center = (bbox[1] + bbox[3] / 2) / row['height']
                width = bbox[2] / row['width']
                height = bbox[3] / row['height']

                f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

    # 원본 val 데이터
    for file_name in tqdm(orig_val, desc="원본 Val 데이터"):
        img_annotations = df_larger[df_larger['file_name'] == file_name]

        # 이미지 복사
        src_path = img_annotations.iloc[0]['image_path']
        dst_path = os.path.join(output_dir, 'val', 'images', file_name)
        if not os.path.exists(dst_path):
            shutil.copy2(src_path, dst_path)

        # 라벨 파일 생성
        label_path = os.path.join(output_dir, 'val', 'labels', os.path.splitext(file_name)[0] + '.txt')
        with open(label_path, 'w') as f:
            for _, row in img_annotations.iterrows():
                class_id = class_to_idx[row['dl_name']]
                bbox = row['bbox']

                # YOLO 형식으로 변환
                x_center = (bbox[0] + bbox[2] / 2) / row['width']
                y_center = (bbox[1] + bbox[3] / 2) / row['height']
                width = bbox[2] / row['width']
                height = bbox[3] / row['height']

                f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

    # 2) 수도 라벨 데이터를 train에 추가 (대부분)
    if selected_pseudo_images:
        pseudo_train, pseudo_val = train_test_split(selected_pseudo_images, test_size=0.1, random_state=42)

        print(f"수도 라벨 분할: Train {len(pseudo_train)}장, Val {len(pseudo_val)}장")

        # 수도 라벨 train 데이터
        for img_name in tqdm(pseudo_train, desc="수도라벨 Train"):
            # 이미지 복사
            src_img = os.path.join(pseudo_reduced_dir, 'images', img_name)
            dst_img = os.path.join(output_dir, 'train', 'images', img_name)
            if os.path.exists(src_img) and not os.path.exists(dst_img):
                shutil.copy2(src_img, dst_img)

            # 라벨 복사
            label_name = os.path.splitext(img_name)[0] + '.txt'
            src_label = os.path.join(pseudo_reduced_dir, 'labels', label_name)
            dst_label = os.path.join(output_dir, 'train', 'labels', label_name)
            if os.path.exists(src_label) and not os.path.exists(dst_label):
                shutil.copy2(src_label, dst_label)

        # 수도 라벨 val 데이터
        for img_name in tqdm(pseudo_val, desc="수도라벨 Val"):
            # 이미지 복사
            src_img = os.path.join(pseudo_reduced_dir, 'images', img_name)
            dst_img = os.path.join(output_dir, 'val', 'images', img_name)
            if os.path.exists(src_img) and not os.path.exists(dst_img):
                shutil.copy2(src_img, dst_img)

            # 라벨 복사
            label_name = os.path.splitext(img_name)[0] + '.txt'
            src_label = os.path.join(pseudo_reduced_dir, 'labels', label_name)
            dst_label = os.path.join(output_dir, 'val', 'labels', label_name)
            if os.path.exists(src_label) and not os.path.exists(dst_label):
                shutil.copy2(src_label, dst_label)

    # 최종 통계
    train_images = len(glob.glob(os.path.join(output_dir, 'train', 'images', '*')))
    val_images = len(glob.glob(os.path.join(output_dir, 'val', 'images', '*')))

    print(f"더 큰 데이터셋 생성 완료:")
    print(f"  Train: {train_images}장")
    print(f"  Val: {val_images}장")
    print(f"  총합: {train_images + val_images}장")

# 더 큰 YOLO 데이터셋 생성
larger_dataset_dir = './pill_dataset_larger'
create_larger_yolo_dataset(df_larger, selected_pseudo_images, pseudo_reduced_dir, larger_dataset_dir, class_to_idx)

# YAML 파일 생성
dataset_info = {
    'path': os.path.abspath(larger_dataset_dir),
    'train': 'train/images',
    'val': 'val/images',
    'names': idx_to_class
}

yaml_path = './pill_dataset_larger.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(dataset_info, f)

print(f"YAML 파일 생성: {yaml_path}")
clear_memory()

# 4. 더 큰 데이터로 모델 학습
print("\n=== 더 큰 데이터셋으로 모델 학습 ===")
model = YOLO('yolov8n.pt')
model.train(
    data=yaml_path,
    epochs=25,        # 더 많은 에포크
    patience=12,
    batch=6,          # CPU를 고려해 조금 작게
    imgsz=640,        # 더 높은 해상도
    workers=2,
    lr0=0.01,         # 학습률 조정
    project='larger_models',
    name='final_larger',
    verbose=True,
    save_period=5
)

# 학습 후 메모리 정리
del model
clear_memory()

# 5. mAP 측정
print("\n=== mAP 측정 ===")
trained_model = YOLO('larger_models/final_larger/weights/best.pt')

results = trained_model.val(
    data=yaml_path,
    imgsz=640,
    conf=0.25,
    iou=0.5,
    batch=4,
    verbose=True,
    workers=2
)

print("\n=== 최종 성능 결과 ===")
print(f"mAP@0.5: {results.box.map50:.4f}")
print(f"mAP@0.5:0.95: {results.box.map:.4f}")
print(f"Precision: {results.box.mp:.4f}")
print(f"Recall: {results.box.mr:.4f}")

# 데이터셋 요약
print(f"\n=== 데이터셋 요약 ===")
print(f"완전한 어노테이션: 2,406개 → {len(df_larger)}개 (700개 줄임)")
print(f"수도 라벨 데이터: 약 1,445개 사용")
print(f"총 학습 데이터: 약 {len(df_larger) + 1445}개")
print(f"이전 결과(1,400개): mAP@0.5 = 44.52%")
print(f"현재 결과({len(df_larger) + 1445}개): mAP@0.5 = {results.box.map50:.2%}")

improvement = (results.box.map50 - 0.4452) * 100
if improvement > 0:
    print(f"성능 향상: +{improvement:.1f}%p")
else:
    print(f"성능 변화: {improvement:.1f}%p")

del trained_model
clear_memory()
print("\n완료!")

=== 올바른 데이터 축소 (700개씩 줄이기) ===
현재 df_complete: 2406개
원본 완전한 어노테이션 데이터: 2406개
원본 이미지 수: 639장
이미지당 평균 객체 수: 3.77개
제거할 이미지 수: 185장
축소 후 데이터: 1708개 (454장 이미지)
실제 줄어든 양: 698개
클래스 수: 73개
현재 축소된 수도 라벨 이미지: 0장
현재 총 수도 라벨 수: 0개
현재 데이터(0개)가 목표(1445개)보다 적습니다.
모든 데이터를 사용합니다.
더 큰 YOLO 데이터셋 생성 중...
원본 데이터 분할: Train 363장, Val 91장


원본 Val 데이터: 100%|██████████| 91/91 [00:01<00:00, 61.33it/s]


더 큰 데이터셋 생성 완료:
  Train: 363장
  Val: 91장
  총합: 454장
YAML 파일 생성: ./pill_dataset_larger.yaml

=== 더 큰 데이터셋으로 모델 학습 ===
Ultralytics 8.3.201 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=6, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./pill_dataset_larger.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=final_larger, nbs=64, 

# 매핑

In [ ]:
from google.colab import drive, files
import json
from ultralytics import YOLO
import glob
from tqdm import tqdm

# Drive 마운트
drive.mount('/content/drive')

# 모델 로드
model_path = '/content/drive/MyDrive/pill_project/ensemble_models/model2_large/weights/best.pt'
model = YOLO(model_path)

# 모델의 클래스 이름
class_names = model.names
print(f"모델 클래스 개수: {len(class_names)}")

# JSON에서 dl_name -> category_id 매핑 추출 (모든 파일 검사!)
json_dir = '/content/drive/MyDrive/pill_project/train_annotations'
json_files = glob.glob(json_dir + '/**/*.json', recursive=True)

print(f"총 {len(json_files)}개 JSON 파일 검사 중...")

name_to_cat_id = {}
for json_file in tqdm(json_files):  # 모든 파일 검사
    try:
        with open(json_file, 'r') as f:
            data = json.load(f)
        if 'images' in data and data['images']:
            img_info = data['images'][0]
            if 'dl_name' in img_info and 'annotations' in data and data['annotations']:
                name = img_info['dl_name']
                cat_id = data['annotations'][0]['category_id']
                name_to_cat_id[name] = cat_id
    except:
        continue

    # 충분히 모았으면 중단
    if len(name_to_cat_id) >= 73:
        break

print(f"발견된 클래스: {len(name_to_cat_id)}개")

# YOLO 인덱스 -> category_id 매핑
yolo_idx_to_original_cat_id = {i: name_to_cat_id[name] for i, name in class_names.items() if name in name_to_cat_id}

print(f"\n최종 매핑: {len(yolo_idx_to_original_cat_id)}개")
print(f"누락된 클래스: {73 - len(yolo_idx_to_original_cat_id)}개")

# 저장
with open('class_mapping.json', 'w') as f:
    json.dump(yolo_idx_to_original_cat_id, f)

files.download('class_mapping.json')

# 이 코드로 확인
print(f"모델의 클래스 개수: {len(model.names)}")
print(f"매핑된 클래스 개수: {len(yolo_idx_to_original_cat_id)}")
print(f"모델 클래스 범위: 0 ~ {len(model.names)-1}")
print(f"매핑 키 범위: {min(yolo_idx_to_original_cat_id.keys())} ~ {max(yolo_idx_to_original_cat_id.keys())}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
모델 클래스 개수: 73
총 4526개 JSON 파일 검사 중...


 94%|█████████▍| 4262/4526 [00:06<00:00, 621.73it/s]

발견된 클래스: 73개

최종 매핑: 73개
누락된 클래스: 0개


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

모델의 클래스 개수: 73
매핑된 클래스 개수: 73
모델 클래스 범위: 0 ~ 72
매핑 키 범위: 0 ~ 72
